## Anaconda Euler

In [1]:
import io
import itertools
import functools
import requests

import numpy as np
from scipy import linalg, special, optimize
import pandas as pd
import sympy
import numba

import num2words

%cd ~/Stat/test

/Users/fangda/Stat/test


In [161]:
%%time
## Problem 1: Multiples of 3 and 5

n_list = [3, 5]
n_max = 1000

print(np.unique(np.concatenate([np.arange(0, n_max, i) for i in n_list])).sum())

233168
CPU times: user 493 µs, sys: 498 µs, total: 991 µs
Wall time: 595 µs


In [160]:
%%time
## Problem 2: Even Fibonacci numbers

n_max = 4e6

phi = (1 + np.sqrt(5)) / 2
def fibonacci(n, n_init=0):
    n_list = n_init + np.arange(n)
    return(((phi ** n_list - (1 - phi) ** n_list) / np.sqrt(5)).astype('int'))

i_max = int(np.log(4e6) / np.log(phi)) + 1
print(sum([i for i in fibonacci(i_max, n_init=2) if (i < n_max) & (i % 2 == 0)]))

4613732
CPU times: user 369 µs, sys: 165 µs, total: 534 µs
Wall time: 429 µs


In [159]:
%%time
## Problem 3: Largest prime factor

n = 600851475143

print(max(sympy.factorint(n).keys()))

6857
CPU times: user 602 µs, sys: 218 µs, total: 820 µs
Wall time: 679 µs


In [158]:
%%time
## Problem 4: Largest palindrome product

n_min = 100
n_max = 1000

S = pd.Series(np.unique(np.outer(np.arange(n_min, n_max), np.arange(n_min, n_max))).astype("str"))
print(int(S[S == S.str[::-1]].iloc[-1]))

906609
CPU times: user 230 ms, sys: 36 ms, total: 266 ms
Wall time: 271 ms


In [157]:
%%time
## Problem 5: Smallest multiple

n_list = range(1, 21)

S = pd.concat([pd.Series(sympy.factorint(i)) for i in n_list], axis=1).fillna(0).max(axis=1).astype("int")
print(np.prod(S.index.values ** S.values))

232792560
CPU times: user 14 ms, sys: 1.58 ms, total: 15.6 ms
Wall time: 16.3 ms


In [156]:
%%time
## Problem 6: Sum square difference

n_list = 1 + np.arange(100)

print(sum(n_list) ** 2 - sum(n_list ** 2))

25164150
CPU times: user 339 µs, sys: 194 µs, total: 533 µs
Wall time: 368 µs


In [155]:
%%time
## Problem 7: 10001st prime

n = 10001

print(sympy.prime(n))

104743
CPU times: user 57.5 ms, sys: 9.18 ms, total: 66.7 ms
Wall time: 65.1 ms


In [154]:
%%time
## Problem 8: Largest product in a series

n = 13
s = '''73167176531330624919225119674426574742355349194934
96983520312774506326239578318016984801869478851843
85861560789112949495459501737958331952853208805511
12540698747158523863050715693290963295227443043557
66896648950445244523161731856403098711121722383113
62229893423380308135336276614282806444486645238749
30358907296290491560440772390713810515859307960866
70172427121883998797908792274921901699720888093776
65727333001053367881220235421809751254540594752243
52584907711670556013604839586446706324415722155397
53697817977846174064955149290862569321978468622482
83972241375657056057490261407972968652414535100474
82166370484403199890008895243450658541227588666881
16427171479924442928230863465674813919123162824586
17866458359124566529476545682848912883142607690042
24219022671055626321111109370544217506941658960408
07198403850962455444362981230987879927244284909188
84580156166097919133875499200524063689912560717606
05886116467109405077541002256983155200055935729725
71636269561882670428252483600823257530420752963450'''.replace("\n", "")

print(pd.DataFrame([list(s[i:(i+13)]) for i in range(len(s) - n)]).astype("int").prod(axis=1).max())

23514624000
CPU times: user 6.45 ms, sys: 3.82 ms, total: 10.3 ms
Wall time: 9.93 ms


In [153]:
%%time
## Problem 9: Special Pythagorean triplet

n = 1000

a_list = np.arange(n / 2)
b_list = np.arange(n / 2)
c_list = n - np.add.outer(a_list, b_list)

a, b = np.where(c_list ** 2 == np.add.outer(a_list ** 2, b_list ** 2))[0]
print(a * b * (n - a - b))

31875000
CPU times: user 4.09 ms, sys: 2.92 ms, total: 7.01 ms
Wall time: 5.54 ms


In [152]:
%%time
## Problem 10: Summation of primes

n_max = 2e6

print(sum(sympy.primerange(1, n_max)))

142913828922
CPU times: user 10.2 s, sys: 26.2 ms, total: 10.3 s
Wall time: 10.3 s


In [151]:
%%time
## Problem 11: Largest product in a grid

n = 4
A = np.loadtxt(io.StringIO('''08 02 22 97 38 15 00 40 00 75 04 05 07 78 52 12 50 77 91 08
49 49 99 40 17 81 18 57 60 87 17 40 98 43 69 48 04 56 62 00
81 49 31 73 55 79 14 29 93 71 40 67 53 88 30 03 49 13 36 65
52 70 95 23 04 60 11 42 69 24 68 56 01 32 56 71 37 02 36 91
22 31 16 71 51 67 63 89 41 92 36 54 22 40 40 28 66 33 13 80
24 47 32 60 99 03 45 02 44 75 33 53 78 36 84 20 35 17 12 50
32 98 81 28 64 23 67 10 26 38 40 67 59 54 70 66 18 38 64 70
67 26 20 68 02 62 12 20 95 63 94 39 63 08 40 91 66 49 94 21
24 55 58 05 66 73 99 26 97 17 78 78 96 83 14 88 34 89 63 72
21 36 23 09 75 00 76 44 20 45 35 14 00 61 33 97 34 31 33 95
78 17 53 28 22 75 31 67 15 94 03 80 04 62 16 14 09 53 56 92
16 39 05 42 96 35 31 47 55 58 88 24 00 17 54 24 36 29 85 57
86 56 00 48 35 71 89 07 05 44 44 37 44 60 21 58 51 54 17 58
19 80 81 68 05 94 47 69 28 73 92 13 86 52 17 77 04 89 55 40
04 52 08 83 97 35 99 16 07 97 57 32 16 26 26 79 33 27 98 66
88 36 68 87 57 62 20 72 03 46 33 67 46 55 12 32 63 93 53 69
04 42 16 73 38 25 39 11 24 94 72 18 08 46 29 32 40 62 76 36
20 69 36 41 72 30 23 88 34 62 99 69 82 67 59 85 74 04 36 16
20 73 35 29 78 31 90 01 74 31 49 71 48 86 81 16 23 57 05 54
01 70 54 71 83 51 54 69 16 92 33 48 61 43 52 01 89 19 67 48'''), dtype = 'int')

A_pad = np.pad(A, [[1, 1], [1, 1]], mode="constant", constant_values=0)
max_v = np.max(functools.reduce(np.multiply, [A_pad[i:-(n - i)] for i in range(n)]))
max_h = np.max(functools.reduce(np.multiply, [A_pad[:, i:-(n - i)] for i in range(n)]))
max_d = np.max(functools.reduce(np.multiply, [A_pad[i:-(n - i), i:-(n - i)] for i in range(n)]))
max_ad = np.max(functools.reduce(np.multiply, [A_pad[(n - i):-(i + 1), i:-(n - i)] for i in range(n)]))
print(max(max_v, max_h, max_d, max_ad))

70600674
CPU times: user 1.47 ms, sys: 502 µs, total: 1.97 ms
Wall time: 1.93 ms


In [150]:
%%time
## Problem 12: Highly divisible triangular number

n = 500

i_val = 0
for i in itertools.count():
    i_val += i
    if len(sympy.divisors(i_val)) > 500:
        break
print(i_val)

76576500
CPU times: user 1.11 s, sys: 7.98 ms, total: 1.12 s
Wall time: 1.12 s


In [149]:
%%time
## Problem 13: Large sum

n = 10
n_list = np.loadtxt(io.StringIO('''37107287533902102798797998220837590246510135740250
46376937677490009712648124896970078050417018260538
74324986199524741059474233309513058123726617309629
91942213363574161572522430563301811072406154908250
23067588207539346171171980310421047513778063246676
89261670696623633820136378418383684178734361726757
28112879812849979408065481931592621691275889832738
44274228917432520321923589422876796487670272189318
47451445736001306439091167216856844588711603153276
70386486105843025439939619828917593665686757934951
62176457141856560629502157223196586755079324193331
64906352462741904929101432445813822663347944758178
92575867718337217661963751590579239728245598838407
58203565325359399008402633568948830189458628227828
80181199384826282014278194139940567587151170094390
35398664372827112653829987240784473053190104293586
86515506006295864861532075273371959191420517255829
71693888707715466499115593487603532921714970056938
54370070576826684624621495650076471787294438377604
53282654108756828443191190634694037855217779295145
36123272525000296071075082563815656710885258350721
45876576172410976447339110607218265236877223636045
17423706905851860660448207621209813287860733969412
81142660418086830619328460811191061556940512689692
51934325451728388641918047049293215058642563049483
62467221648435076201727918039944693004732956340691
15732444386908125794514089057706229429197107928209
55037687525678773091862540744969844508330393682126
18336384825330154686196124348767681297534375946515
80386287592878490201521685554828717201219257766954
78182833757993103614740356856449095527097864797581
16726320100436897842553539920931837441497806860984
48403098129077791799088218795327364475675590848030
87086987551392711854517078544161852424320693150332
59959406895756536782107074926966537676326235447210
69793950679652694742597709739166693763042633987085
41052684708299085211399427365734116182760315001271
65378607361501080857009149939512557028198746004375
35829035317434717326932123578154982629742552737307
94953759765105305946966067683156574377167401875275
88902802571733229619176668713819931811048770190271
25267680276078003013678680992525463401061632866526
36270218540497705585629946580636237993140746255962
24074486908231174977792365466257246923322810917141
91430288197103288597806669760892938638285025333403
34413065578016127815921815005561868836468420090470
23053081172816430487623791969842487255036638784583
11487696932154902810424020138335124462181441773470
63783299490636259666498587618221225225512486764533
67720186971698544312419572409913959008952310058822
95548255300263520781532296796249481641953868218774
76085327132285723110424803456124867697064507995236
37774242535411291684276865538926205024910326572967
23701913275725675285653248258265463092207058596522
29798860272258331913126375147341994889534765745501
18495701454879288984856827726077713721403798879715
38298203783031473527721580348144513491373226651381
34829543829199918180278916522431027392251122869539
40957953066405232632538044100059654939159879593635
29746152185502371307642255121183693803580388584903
41698116222072977186158236678424689157993532961922
62467957194401269043877107275048102390895523597457
23189706772547915061505504953922979530901129967519
86188088225875314529584099251203829009407770775672
11306739708304724483816533873502340845647058077308
82959174767140363198008187129011875491310547126581
97623331044818386269515456334926366572897563400500
42846280183517070527831839425882145521227251250327
55121603546981200581762165212827652751691296897789
32238195734329339946437501907836945765883352399886
75506164965184775180738168837861091527357929701337
62177842752192623401942399639168044983993173312731
32924185707147349566916674687634660915035914677504
99518671430235219628894890102423325116913619626622
73267460800591547471830798392868535206946944540724
76841822524674417161514036427982273348055556214818
97142617910342598647204516893989422179826088076852
87783646182799346313767754307809363333018982642090
10848802521674670883215120185883543223812876952786
71329612474782464538636993009049310363619763878039
62184073572399794223406235393808339651327408011116
66627891981488087797941876876144230030984490851411
60661826293682836764744779239180335110989069790714
85786944089552990653640447425576083659976645795096
66024396409905389607120198219976047599490197230297
64913982680032973156037120041377903785566085089252
16730939319872750275468906903707539413042652315011
94809377245048795150954100921645863754710598436791
78639167021187492431995700641917969777599028300699
15368713711936614952811305876380278410754449733078
40789923115535562561142322423255033685442488917353
44889911501440648020369068063960672322193204149535
41503128880339536053299340368006977710650566631954
81234880673210146739058568557934581403627822703280
82616570773948327592232845941706525094512325230608
22918802058777319719839450180888072429661980811197
77158542502016545090413245809786882778948721859617
72107838435069186155435662884062257473692284509516
20849603980134001723930671666823555245252804609722
53503534226472524250874054075591789781264330331690'''))

out = sum(n_list)
print(int(out / 10 ** int(np.log10(out) - n + 1)))

5537376230
CPU times: user 1.39 ms, sys: 290 µs, total: 1.68 ms
Wall time: 1.73 ms


In [148]:
%%time
## Problem 14: Longest Collatz sequence

n_min = 2
n_max = int(1e6)

chain_val_dict = {}
chain_len_dict = {1: 0}
def chain(x):
    if x != 1:
        x_next = chain_val_dict.get(x)
        if x_next is None:
            if x % 2 == 1:
                x_next = 3 * x + 1
            else:
                x_next = x // 2
            chain_val_dict[x] = x_next
            chain(x_next)
        chain_len_dict[x] = chain_len_dict[x_next] + 1

[chain(i) for i in range(n_min, n_max)]
print(pd.Series(chain_len_dict)[:n_max].argmax())

837799
CPU times: user 6.45 s, sys: 221 ms, total: 6.67 s
Wall time: 6.69 s


In [147]:
%%time
## Problem 15: Lattice paths

n = 20

print(special.comb(2 * n, n, exact=True))

137846528820
CPU times: user 414 µs, sys: 449 µs, total: 863 µs
Wall time: 788 µs


In [146]:
%%time
## Problem 16: Power digit sum

n = 2 ** 1000

print(np.array(list(str(n)), dtype="int").sum())

1366
CPU times: user 589 µs, sys: 385 µs, total: 974 µs
Wall time: 644 µs


In [145]:
%%time
## Problem 17: Number letter counts

n_max = 1000

print(len("".join([num2words.num2words(i) for i in 1 + np.arange(n_max)]).replace(" ", "").replace("-", "")))

21124
CPU times: user 1.89 s, sys: 12 ms, total: 1.9 s
Wall time: 1.92 s


In [144]:
%%time
## Problem 18: Maximum path sum I

A = pd.read_fwf(io.StringIO('''75
95 64
17 47 82
18 35 87 10
20 04 82 47 65
19 01 23 75 03 34
88 02 77 73 07 63 67
99 65 04 28 06 16 70 92
41 41 26 56 83 40 80 70 33
41 48 72 33 47 32 37 16 94 29
53 71 44 65 25 43 91 52 97 51 14
70 11 33 28 77 73 17 78 39 68 17 57
91 71 52 38 17 14 91 43 58 50 27 29 48
63 66 04 68 89 53 67 30 73 16 69 87 40 31
04 62 98 27 23 09 70 98 73 93 38 53 60 04 23'''), delim_whitespace=True, header=None).fillna(0).values.astype('int')

A_cum = A.copy()
A_choice = np.zeros(A.shape, dtype="int")
for i in np.arange(len(A)-1, 0, -1):
    A_choice[i - 1, :-1] = (np.diff(A_cum[i]) > 0).astype("int")
    A_cum[i - 1] += A_cum[i, np.arange(len(A)) + A_choice[i - 1]]
print(A_cum[0, 0])

1074
CPU times: user 4.92 ms, sys: 4.67 ms, total: 9.6 ms
Wall time: 9.3 ms


In [143]:
%%time
## Problem 19: Counting Sundays

date_begin = "1901-01-01"
date_end = "2000-12-31"

S = pd.Series(pd.date_range(date_begin, date_end, freq="MS").strftime('%A'))
print(S.value_counts()["Sunday"])

171
CPU times: user 76.6 ms, sys: 3 ms, total: 79.6 ms
Wall time: 83.1 ms


In [142]:
%%time
## Problem 20: Factorial digit sum

n = 100

print(np.array(list(str(special.factorial(n, exact=True))), dtype="int").sum())

648
CPU times: user 570 µs, sys: 347 µs, total: 917 µs
Wall time: 576 µs


In [141]:
%%time
## Problem 21: Amicable numbers

n_max = 10000

def sum_divisors(x):
    return(sum(sympy.divisors(x)[:-1]))

S = pd.Series([sum_divisors(i) for i in np.arange(n_max)])
print(S.index.values[(S.index == S.loc[S.values]) & (S.index != S.values)].sum())

31626
CPU times: user 323 ms, sys: 3.42 ms, total: 326 ms
Wall time: 332 ms


In [140]:
%%time
## Problem 22: Names scores

df = pd.read_csv("https://projecteuler.net/project/resources/p022_names.txt").T

S = df.sort_index().reset_index()["index"]
print((S.apply(lambda x: sum([ord(i) - 64 for i in list(x)])) * (S.index.values + 1)).sum())

871198282
CPU times: user 1.73 s, sys: 27.6 ms, total: 1.75 s
Wall time: 2.71 s


In [139]:
%%time
## Problem 23: Non-abundant sums

n_max = 28123

def sum_divisors(x):
    return(sum(sympy.divisors(x)[:-1]))

S = pd.Series([sum_divisors(i) for i in np.arange(n_max)])
n_list = S.index.values[S.index < S.values]
print(S.index.difference(np.unique(np.add.outer(n_list, n_list))).values.sum())

4179871
CPU times: user 2.85 s, sys: 318 ms, total: 3.17 s
Wall time: 3.26 s


In [138]:
%%time
## Problem 24: Lexicographic permutations

n = 10
n_th = int(1e6)

n_list = list(range(n))
out = 0
n_th -= 1
for i in reversed(range(n)):
    i_val = special.factorial(i, exact=True)
    out += n_list.pop(n_th // i_val) * 10 ** i
    n_th %= i_val
print(out)

2783915460
CPU times: user 209 µs, sys: 89 µs, total: 298 µs
Wall time: 247 µs


In [137]:
%%time
## Problem 25: 1000-digit Fibonacci number

n = 1000

i = 3
i_val = 2
phi = (1 + np.sqrt(5)) / 2
print(int(np.ceil((n - 1 - np.log10(i_val)) / np.log10(phi) + i)))

4782
CPU times: user 442 µs, sys: 320 µs, total: 762 µs
Wall time: 489 µs


In [136]:
%%time
## Problem 26: Reciprocal cycles

n_max = 1000

out = []
for i in range(1, n_max):
    out.append(list(sympy.cycle_length(lambda x: 10 * x % i, 1))[0][0])
print(np.argmax(out) + 1)

983
CPU times: user 107 ms, sys: 2.68 ms, total: 110 ms
Wall time: 108 ms


In [135]:
%%time
## Problem 27: Quadratic primes

a_bound = 1000

a_list = np.arange(-a_bound, a_bound + 1)
A_choice = np.ones((len(a_list), len(a_list)), dtype="bool")

for n in itertools.count():
    A = n ** 2 + np.add.outer(n * a_list, a_list)
    A_update = [sympy.isprime(i) for i in A[A_choice]]
    if np.any(A_update) == False:
        break
    A_choice[A_choice] = A_update
print(np.prod([a_list[i[0]] for i in np.where(A_choice)]))

-59231
CPU times: user 16.9 s, sys: 756 ms, total: 17.7 s
Wall time: 17.8 s


In [134]:
%%time
## Problem 28: Number spiral diagonals

n = 1001

print(4 * np.cumsum(np.insert(5 + np.arange((n + 1) // 2 - 1) * 8, 0, 1)).sum() - 3)

669171001
CPU times: user 421 µs, sys: 222 µs, total: 643 µs
Wall time: 486 µs


In [133]:
%%time
## Problem 29: Distinct powers

n_min = 2
n_max = 100

n_list = np.arange(n_min, n_max + 1, dtype="float")
print(len(np.unique(np.power.outer(n_list, n_list))))

9183
CPU times: user 1.78 ms, sys: 1.17 ms, total: 2.94 ms
Wall time: 1.38 ms


In [132]:
%%time
## Problem 30: Digit fifth powers

n = 5

n_list = np.arange(2, 9 ** n * n)
S = pd.Series(functools.reduce(np.add, [(n_list // (10 ** i) % 10) ** n for i in np.arange(n + 1)]), n_list)
print(sum(S.index[S == S.index].values))

443839
CPU times: user 60.6 ms, sys: 9.38 ms, total: 70 ms
Wall time: 68.9 ms


In [131]:
%%time
## Problem 31: Coin sums

n = 200
n_list = [1, 2, 5, 10, 20, 50, 100, 200]

def count_coin(x, val):
    if x == 0:
        return([[val]])
    out = []
    for i in range(val // n_list[x] + 1):
        out += [j + [i] for j in count_coin(x - 1, val - n_list[x] * i)]
    return(out)
print(len(count_coin(len(n_list) - 1, 200)))

73682
CPU times: user 481 ms, sys: 13.5 ms, total: 494 ms
Wall time: 500 ms


In [130]:
%%time
## Problem 32: Pandigital products

n_list = np.arange(1, 10)

out = []
for i in n_list:
    for j in n_list:
        res_list = np.setdiff1d(n_list, [i, j, (i * j) % 10])
        if len(res_list) == 6:
            for k in itertools.permutations(res_list, 3):
                k_val_0 = (i + k[0] * 10 + k[1] * 100 + k[2] * 1000) * j
                k_val_1 = (i + k[0] * 10 + k[1] * 100) * (j + k[2] * 10)
                prod_list = list(np.setdiff1d(res_list, k))
                if list(np.sort([int(x) for x in str(k_val_0)[:-1]])) == prod_list:
                    out.append(k_val_0)
                if list(np.sort([int(x) for x in str(k_val_1)[:-1]])) == prod_list:
                    out.append(k_val_1)
print(sum(np.unique(out)))

45228
CPU times: user 430 ms, sys: 3.23 ms, total: 434 ms
Wall time: 439 ms


In [129]:
%%time
## Problem 33: Digit cancelling fractions

out = []
for i in range(1, 10):
    for j in range(1, i):
        i_val = sympy.Rational(j, i)
        for k in range(1, 10):
            if sympy.Rational(10 * k + j, 10 * i + k) == i_val:
                out.append(i_val)
            if sympy.Rational(10 * j + k, 10 * k + i) == i_val:
                out.append(i_val)
print(sympy.prod(out).q)

100
CPU times: user 14.5 ms, sys: 1.92 ms, total: 16.5 ms
Wall time: 15.5 ms


In [128]:
%%time
## Problem 34: Digit factorials

n_min = 10

n_max = special.factorial(9, exact=True)
n_dict = {str(i): special.factorial(i, exact=True) for i in range(10)}
print(sum(i for i in range(n_min, n_max) if sum(n_dict[j] for j in str(i)) == i))

40730
CPU times: user 618 ms, sys: 3.27 ms, total: 621 ms
Wall time: 626 ms


In [127]:
%%time
## Problem 35: Circular primes

n_max = int(1e6)

n_list = set(sympy.primerange(1, n_max))
out = []
for i in n_list:
    i_len = 1 + int(np.log10(i))
    i_val = set(int((str(i) * 2)[j:(j+i_len)]) for j in range(i_len))
    if i_val.issubset(n_list):
        out.append(i)
print(len(out))

55
CPU times: user 5.95 s, sys: 16.7 ms, total: 5.97 s
Wall time: 5.99 s


In [126]:
%%time
## Problem 36: Double-base palindromes

n_max = 1000

out = []
for i in range(1, n_max):
    i_val_0 = int(str(i)[:-1] + str(i)[::-1])
    i_val_1 = int(str(i) + str(i)[::-1])
    i_val_0_base2 = "{0:b}".format(i_val_0)
    i_val_1_base2 = "{0:b}".format(i_val_1)
    if i_val_0_base2 == i_val_0_base2[::-1]:
        out.append(i_val_0)
    if i_val_1_base2 == i_val_1_base2[::-1]:
        out.append(i_val_1)
print(sum(out))

872187
CPU times: user 4.97 ms, sys: 374 µs, total: 5.34 ms
Wall time: 5.12 ms


In [125]:
%%time
## Problem 37: Truncatable primes

n_min = 10
n_stop = 11

n_max = int(1e9)
n_list = sympy.primerange(n_min, n_max)
out = []
left_list = {2, 3, 5, 7}
right_list = {2, 3, 5, 7}
for i in n_list:
    if i // 10 in left_list:
        left_list.add(i)
    if int(str(i)[1:]) in right_list:
        right_list.add(i)
        if i // 10 in left_list:
            out.append(i)
            if len(out) >= n_stop:
                break
print(sum(out))

748317
CPU times: user 3.75 s, sys: 9.7 ms, total: 3.76 s
Wall time: 3.78 s


In [124]:
%%time
## Problem 38: Pandigital multiples

n_list = np.arange(1, 10)

out = []
for i in [4]:
    for j in itertools.permutations(n_list, i):
        res_list = np.setdiff1d(n_list, j)
        i_val = int("".join([str(k) for k in j]))
        i_val_list = [int(k) for k in str(2 * i_val)]
        if list(np.sort(i_val_list)) == list(res_list):
            out.append(int("".join([str(k) for k in list(j) + i_val_list])))
print(max(out))

932718654
CPU times: user 260 ms, sys: 1.95 ms, total: 262 ms
Wall time: 268 ms


In [123]:
%%time
## Problem 39: Integer right triangles

n_max = 1000
n_min = 10

n_list = np.arange(n_min, n_max + 1)
out = []
for i in n_list:
    a_list = np.arange(1, i // 2)
    A = (i - np.add.outer(a_list, a_list)) ** 2 == np.add.outer(a_list ** 2, a_list ** 2)
    out.append(np.sum(A))
print(n_list[np.argmax(out)])

840
CPU times: user 737 ms, sys: 352 ms, total: 1.09 s
Wall time: 1.1 s


In [122]:
%%time
## Problem 39: Integer right triangles
### Alternative solve 1

n_max = 1000
n_min = 10

@numba.jit
def right_triangle_solve(p, a, b):
    return(a ** 2 + b ** 2 - (p - a - b) ** 2)

n_list = np.arange(n_min, n_max + 1)
out = []
for i in n_list:
    b = (i - 1) // 2
    count = 0
    for a in np.arange(1, i // 2):
        b = round(optimize.brentq(lambda x: right_triangle_solve(i, a, x), b - 3, b + 1, xtol = 0.5))
        if right_triangle_solve(i, a, b) == 0:
            count += 1
    out.append(count)
print(n_list[np.argmax(out)])

840
CPU times: user 1.91 s, sys: 129 ms, total: 2.04 s
Wall time: 2.19 s


In [121]:
%%time
## Problem 40: Champernowne's constant

n_max = int(1e6)
n_list = [1, 10, 100, 1000, 10000, 100000, 1000000]

s = "".join([str(i) for i in range(n_max)])
print(np.prod([int(s[i]) for i in n_list]))

210
CPU times: user 329 ms, sys: 29.8 ms, total: 359 ms
Wall time: 364 ms


In [120]:
%%time
## Problem 41: Pandigital prime

n_list = range(1, 10)

out = []
for i in n_list:
    for j in itertools.permutations(range(1, i + 1)):
        i_val = int("".join([str(k) for k in j]))
        if sympy.isprime(i_val):
            out.append(i_val)
print(max(out))

7652413
CPU times: user 5.91 s, sys: 14.9 ms, total: 5.92 s
Wall time: 5.95 s


In [119]:
%%time
## Problem 42: Coded triangle numbers

url = requests.get("https://projecteuler.net/project/resources/p042_words.txt")

out = np.bincount([sum(ord(j) - 64 for j in i[1:-1]) for i in url.text.split(",")])
n_list = np.arange(1, np.sqrt(2 * len(out)) + 1, dtype = "int")
n_list = n_list * (n_list + 1) // 2
print(sum(out[n_list[n_list < len(out)]]))

162
CPU times: user 31.3 ms, sys: 11.4 ms, total: 42.6 ms
Wall time: 1.49 s


In [118]:
%%time
## Problem 43: Sub-string divisibility

n_list = range(10)
a_list = [2, 3, 5, 7, 11, 13, 17]

print(sum([int("".join(str(j) for j in i)) for i in itertools.permutations(n_list) 
     if all((i[j + 1] * 100 + i[j + 2] * 10 + i[j + 3]) % j_val == 0 for j, j_val in enumerate(a_list))]))

16695334890
CPU times: user 6.97 s, sys: 14.1 ms, total: 6.98 s
Wall time: 7 s


In [117]:
%%time
## Problem 44: Pentagon numbers

def pentagonal(x):
    return(x * (3 * x - 1) // 2)

def pentagonal_solve():
    n_list = set()
    a_list = []
    for i in itertools.count(1):
        i_val = pentagonal(i)
        n_list.add(pentagonal(2 * i))
        n_list.add(pentagonal(2 * i + 1))
        for j in a_list:
            if ((i_val - j) in n_list) & ((2 * i_val - j) in n_list):
                return(j)
        a_list.append(i_val)
        
print(pentagonal_solve())

5482660
CPU times: user 480 ms, sys: 2.38 ms, total: 482 ms
Wall time: 488 ms


In [116]:
%%time
## Problem 45: Triangular, pentagonal, and hexagonal

n_min = 285

n_list_pen = set()
n_list_hex = set()
for i in itertools.count(2):
    i_val = i * (i + 1) // 2
    n_list_pen.add(i * (3 * i - 1) // 2)
    n_list_hex.add(i * (2 * i - 1))
    if (i > n_min) & (i_val in n_list_pen) & (i_val in n_list_hex):
        break
print(i_val)

1533776805
CPU times: user 84.1 ms, sys: 3.97 ms, total: 88 ms
Wall time: 93.8 ms


In [115]:
%%time
## Problem 46: Goldbach's other conjecture

for i in itertools.count(1):
    i_val = 2 * i + 1
    if not sympy.isprime(i_val):
        i_choice = np.sqrt((i_val - np.array(list(sympy.primerange(1, i_val)))) / 2)
        if (i_choice.round(0) != i_choice).all():
            break
print(i_val)

5777
CPU times: user 12.1 s, sys: 29 ms, total: 12.1 s
Wall time: 12.1 s


In [114]:
%%time
## Problem 47: Distinct primes factors

n = 4

count = 0
for i in itertools.count():
    i_val = len(sympy.primefactors(i))
    if i_val == n:
        count += 1
        if count >= n:
            break
    else:
        count = 0
print(i - 3)

134043
CPU times: user 3.3 s, sys: 10.6 ms, total: 3.31 s
Wall time: 3.33 s


In [113]:
%%time
## Problem 48: Self powers

n_max = 1000
n = 10

def power_last_digits(x, n):
    out = 1
    a = int(10 ** n)
    for i in range(x):
        out *= x
        out %= a
    return(out)
print(sum(power_last_digits(i, n) for i in 1 + np.arange(n_max)) % int(10 ** n))

9110846700
CPU times: user 142 ms, sys: 2.17 ms, total: 144 ms
Wall time: 142 ms


In [112]:
%%time
## Problem 49: Prime permutations

n = 3
n_min = 1000
n_max = 10000
n_exclude = 4817

def prime_perm():
    n_list = list(sympy.primerange(n_min, n_max))
    S = pd.Series([tuple(np.sort(list(str(i)))) for i in n_list], n_list, name="perm")
    for i_key, i in S.reset_index().groupby("perm")["index"]:
        if len(i) >= 3:
            i_val = np.abs(np.add.outer(i, -i))
            for j in range(len(i_val)):
                if (len(np.unique(i_val[j])) < len(i_val)) & (i.values[j] != n_exclude):
                    j_val = pd.Series(i_val[j]).duplicated(keep = False)
                    return(int("".join(str(i.values[k]) for k in [j_val.index[j_val][0], j, j_val.index[j_val][1]])))
print(prime_perm())

296962999629
CPU times: user 71 ms, sys: 819 µs, total: 71.8 ms
Wall time: 74.1 ms


In [111]:
%%time
## Problem 50: Consecutive prime sum

n_max = int(1e6)

n_list = np.array(list(sympy.primerange(1, n_max)))
out = np.zeros(n_max)
for i in range(len(n_list)):
    i_val = n_list[i:].cumsum()
    i_val = i_val[i_val < n_max]
    out[i_val] = np.maximum(out[i_val], 1 + np.arange(len(i_val)))
print(n_list[np.argmax(out[n_list])])

997651
CPU times: user 16.9 s, sys: 89.2 ms, total: 16.9 s
Wall time: 17 s


In [109]:
%%time
## Problem 51: Prime digit replacements

len_min = 8

for n in itertools.count(2):
    n_list = list(sympy.primerange(10 ** (n - 1), 10 ** n))
    df = pd.DataFrame([[int(j) for j in str(i)] for i in n_list], n_list)
    out = []
    for i in range(1, n):
        for j in itertools.combinations(np.arange(1, n), i):
            j_val = np.setdiff1d(np.arange(n), j)
            if len(j_val) == 1:
                for k_key, k in df.groupby(j):
                    if len(k) >= len_min:
                        out.append(min(k.index.values))
            else:
                for k_key, k in df.groupby(j):
                    k_val = (np.diff(k.values[:, j_val], axis=1) == 0).all(axis=1)
                    if sum(k_val) >= len_min:
                        out.append(min(k.index.values[k_val]))
    if out:
        break
print(min(out))

121313
CPU times: user 20 s, sys: 209 ms, total: 20.2 s
Wall time: 20.1 s


In [110]:
%%time
## Problem 52: Permuted multiples

n = 6

def contain_digit(x):
    return(np.sort(list(str(x))))

for i in itertools.count(1):
    i += 10 ** len(str(i))
    i_val = contain_digit(i)
    if all(np.array_equal(i_val, contain_digit(j * i)) for j in np.arange(2, n)):
        break
print(i)

142857
CPU times: user 1.98 s, sys: 8.75 ms, total: 1.99 s
Wall time: 2 s


In [163]:
%%time
## Problem 53: Combinatoric selections

n_max = 100
n_val_max = int(1e6)

out = []
for i in 1 + np.arange(n_max):
    for j in np.arange(i + 1):
        j_val = special.comb(i, j)
        if j_val > n_val_max:
            j = j * 2 - 1
            break
    out.append(i - j)
print(sum(out))

4075
CPU times: user 10.9 ms, sys: 916 µs, total: 11.8 ms
Wall time: 11.1 ms


In [244]:
%%time
## Problem 54: Poker hands

n = 5
n_dict = {"2": 2, "3": 3, "4": 4, "5": 5, "6": 6, "7": 7, "8": 8, "9": 9, "T": 10, "J": 11, "Q": 12, "K": 13, "A": 14}
def poker_rank(df):
    df_rank = pd.DataFrame([np.bincount(i, minlength = 5) for i_key, i in df.iterrows()])
    df_rank[5] = (df_rank[1] == 5) & ((df.diff(axis=1) == 1).sum(axis=1) == 1)
    df_rank = 7 * df_rank[4] + 2 * (df_rank[3] & df_rank[2]) + 4 * df_rank[5] + 3 * df_rank[3] + df_rank[2] + (df + np.arange(15) / 100).idxmax(axis=1) / 100
    return(df_rank)

df = pd.read_fwf("https://projecteuler.net/project/resources/p054_poker.txt", header=None)
df_1 = pd.DataFrame([np.bincount([n_dict[j[0]] for j in i], minlength = 15) for i_key, i in df.iloc[:, :n].iterrows()])
df_2 = pd.DataFrame([np.bincount([n_dict[j[0]] for j in i], minlength = 15) for i_key, i in df.iloc[:, n:].iterrows()])
S_suit = df.iloc[:, :n].apply(lambda x: len(set(i[1] for i in x)) == 1, axis=1).astype('int') - df.iloc[:, n:].apply(lambda x: len(set(i[1] for i in x)) == 1, axis=1).astype('int')
df_rank_1 = poker_rank(df_1)
df_rank_2 = poker_rank(df_2)
print(np.sum(df_rank_1 - df_rank_2 + 5 * S_suit > 0))

376
CPU times: user 494 ms, sys: 4.41 ms, total: 498 ms
Wall time: 985 ms
